# Regression & Clustering #

## Regression --> Pipeline ##
We want to find a function f that that predicts gas consumption (y) based on weather data x1, x2, ..., xn.

What type of regression?  Try using small sample and pick version that (a) minimizes error and (b) gives us small enough runtimes to be able to do it for each consumer ID:
* Linear regression + LASSO
* GLM/nonlinearn (polynomial)
* GLM/nonlinear (poisson)
* Kernel Ridge Regression (KRR)
* SVR
    * RBF
    * Polynomial kernel

How much data to use?  Last year?  All?  
* What is runtime difference? 
* What is accuracy difference?

#### parallel processing in python: 

* https://docs.python.org/3/library/multiprocessing.html 
* see also: Python Data Science Essentials ~pg390


* http://docs.dask.org/en/latest/ 
* http://docs.dask.org/en/latest/scheduling.html 
* http://docs.dask.org/en/latest/setup/single-distributed.html 
* http://docs.dask.org/en/latest/use-cases.html

In [2]:
# imports
import pandas as pd
import numpy as np
import time

#from joblib import Parallel
from dask import compute, delayed
from dask.distributed import Client, LocalCluster 
cl = LocalCluster()
client = Client(cl)
cl
# close

In [4]:
# read pickle
location = '/Users/mithras/Documents/_SCHOOL/_Drexel/BUSN 710 - Capstone/Data/Forecasting Project/'
daily = pd.read_pickle(location+'peco_daily.pkl.zip')
hourly = pd.read_pickle(location+'peco_hourly.pkl.zip')

In [ ]:
# parallelize!  For each ID:

# predict *gas* as a function of weather data
    # how many lag-deltas for each weather variable? (hypothesis <= 3?)
    # additive or multiplicative? (hypothesis: mult)
    # degree polynomial (hypothesis = 3?)

In [ ]:
# linear+LASSO

In [ ]:
# GLM poly

In [ ]:
# GLM poisson

In [ ]:
# KRR

In [ ]:
# SVR RBF

from sklearn.svm import SVC
from sklearn.grid_search import RandomizedSearchCV


hypothesis = SVR(kernel='rbf', random_state=101)
search_dict = {'C': [0.01, 0.1, 1, 10, 100], 
'gamma': [0.1, 0.01, 0.001, 0.0001]}
search_func = RandomizedSearchCV(estimator=hypothesis, 
param_distributions=search_dict, n_iter=10, scoring='accuracy',
n_jobs=-1, iid=True, refit=True, cv=5, random_state=101)
search_func.fit(X_train, y_train)

print ('Best parameters %s' % search_func.best_params_)
print ('Cross validation accuracy: mean = %0.3f' % search_func.best_score_)

In [ ]:
# SVR poly

In [304]:
def errfn (acts, preds, method):
    """
    Calculates error using a variety of functions, 
    incl. Root Mean Squared Error (RMSE), Mean Absolute Error (MAE), & Log Accuracy Ratio (LNQ)
    
    LnQ acts as a symmetric percentage error function.  See: https://papers.ssrn.com/sol3/papers.cfm?abstract_id=2635088

    Parameters
    ----------
    acts : actual values (ground truth) in list or numpy array
    preds : predicted values  in list or numpy array
    method : error function, including RMSE, MAE, LnQ

    Returns
    ----------
    error : float

    @author: Yi Zhu, Alex Graber
    """

    # force method to uppercase
    method = method.upper()
    
    # check to ensure equal length vectors
    if (len(acts) != len(preds)):
        print("Abs and Preds do not have equivalent length!")
    
    # param detection
    if (method == "RMSE"): 
        #calculate RMSE
        from math import sqrt
        from sklearn.metrics import mean_squared_error
        error = sqrt(mean_squared_error(acts, preds))
    elif (method == "MAE"):
        #calculate MAE
        from sklearn.metrics import mean_absolute_error
        error = mean_absolute_error(acts, preds)
    elif (method == "LNQ"):
        #calculate LnQ
        #numerator = abs(acts-preds)
        #denominator = (abs(acts) + abs(preds))/2
        #sum_n = numerator / denominator
        error = np.sum(np.log(preds/acts)**2)
    else:
        print("Attempted method not in [RMSE, MAE, SMAPE]")

    return error


In [ ]:
# create new df with ID, regression weights, regression error

In [5]:
# close dask cluster
cl.close

<bound method LocalCluster.close of LocalCluster('tcp://127.0.0.1:63002', workers=1, ncores=1)>

## Cluster ##

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Sun Jun  3 20:17:26 2018

@author: yizhu6
"""

# import W and reshape the data to get prepared, manually import
nsamples, nx, ny = W_new.shape
data_initial = W_new.reshape((nsamples,nx*ny))

# remove zero obs
df=pd.DataFrame(data_initial)
df['total']= df.sum(axis=1)
df_removezero = df[df.total != 0]
data =np.array(df_removezero.drop(['total'],axis=1))

#########################################

# normalize
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

data_scaled = scaler.fit_transform(data)
    
# determine K using elbow method

from sklearn.cluster import KMeans
from sklearn import metrics
from scipy.spatial.distance import cdist
import matplotlib.pyplot as plt


# euclidean
start_time = time.time()
sse1 = []
K = range(1,12)
for k in K:
    kmeanModel = KMeans(n_clusters=k).fit(data)
    kmeanModel.fit(data)
    sse1.append(sum(np.min(cdist(data, kmeanModel.cluster_centers_, 'euclidean'), axis=1)) / X.shape[0])

# Plot the elbow
plt.plot(K, sse1, 'bx-')
plt.xlabel('Number of K')
plt.ylabel('SSE')
plt.title('The Elbow Method Showing the Optimal K (Euclidean)')
plt.show()
print("--- %s seconds ---" % (time.time() - start_time))

# do clustering
import nltk
from nltk.cluster.kmeans import KMeansClusterer
from sklearn.cluster import AgglomerativeClustering,KMeans

n_clusters = 5

#############################################

#KMeans - Euclidean
kclusterer = KMeansClusterer(n_clusters, distance=nltk.cluster.util.euclidean_distance)
clusters_table = kclusterer.cluster(data, assign_clusters=True)
pd.DataFrame(pd.Series(clusters_table).value_counts(), columns = ['NO. of clients']).T
#                  9   5   7   0   4   3   1   6   2   10  8 
#NO. of clients  3908  13   6   5   4   3   3   3   3   2   1


In [ ]:
# for (k=2 to n):
    # kmeans

In [ ]:
# scree plot

# super awesome visualization and screen plot kernels I came across while doing the NYC taxi fare forcasting project:
# https://www.kaggle.com/ashishpatel26/exploration-of-nyc
# https://www.kaggle.com/willkoehrsen/a-walkthrough-and-a-challenge


In [ ]:
# SSE 
from sklearn.cluster import KMeans
sse = []
K = range(1,20)
for k in K:
    kmeanModel = KMeans(n_clusters=k).fit(data)
    kmeanModel.fit(data)
    sse.append(sum(np.min(cdist(data, kmeanModel.cluster_centers_, 'cosine'), axis=1)) / X.shape[0])


In [ ]:
# def: PCA
    # normalize or not?

pca_2cw = PCA(n_components=2, whiten=True)
X_pca_1cw = pca_2cw.fit_transform(iris.data)
plt.scatter(X_pca_1cw[:,0], X_pca_1cw[:,1], c=iris.target,
alpha=0.8, s=60, marker='o', edgecolors='white')
plt.show()
pca_2cw.explained_variance_ratio_.sum() 

In [ ]:
# visualize cluster identities using PCA on weight vector

In [ ]:
# visualize rate-code identities using PCA on weight vector

## Forecast ##

In [ ]:
def autoARIMA (train, bounds):
    """
    Runs an automatic ARIMA on data, given bounds to speed grid search for p,d,q,P,D,Q params
    
    
    Parameters
    ----------
    train : training data

    Returns
    ----------
    forecast : model

    @author: Yi Zhu, Alex Graber
    """
    
    #preprocessing (since arima takes univariate series as input)
    train.drop('Month',axis=1,inplace=True)
    valid.drop('Month',axis=1,inplace=True)


    #building the model
    from pyramid.arima import auto_arima
    model = auto_arima(train, trace=True, error_action='ignore', suppress_warnings=True)
    model.fit(train)

    forecast = model.predict(n_periods=len(valid))
    forecast = pd.DataFrame(forecast,index = valid.index,columns=['Prediction'])

    return forecast



    # links
    # https://www.analyticsvidhya.com/blog/2018/08/auto-arima-time-series-modeling-python-r/
    # https://machinelearningmastery.com/arima-for-time-series-forecasting-with-python/
    # https://medium.com/@josemarcialportilla/using-python-and-auto-arima-to-forecast-seasonal-time-series-90877adff03c


In [ ]:
# forecast each cluster
fcasts = []
for (i in 1:k):
    # subset data based on cluster ID
    
    
    # split data into training and testing
    ratio = 0.7
    train = data[:int(ratio*(len(data)))]
    test = data[int(ratio*(len(data))):]

    # forecast
    fcast[i]=autoARIMA(train, bounds)
    
    # calculate error
    preds = fcast[i].predictions
    print(errfn(test, preds, "RMSE"))
    print(errfn(test, preds, "SMAPE"))

# compare forecast to testing data
plt.plot(train, label='Train')
plt.plot(test, label='test')
plt.plot(forecast, label='Prediction')
plt.show()










In [ ]:
# forecast each rate code
fcasts = []
for (i in 1:k):
    # subset data based on rate code
    
    
    # split data into training and testing
    ratio = 0.7
    train = data[:int(ratio*(len(data)))]
    test = data[int(ratio*(len(data))):]

    # forecast
    fcast[i]=autoARIMA(train, bounds)
    
    # calculate error
    preds = fcast[i].predictions
    print(errfn(test, preds, "RMSE"))
    print(errfn(test, preds, "SMAPE"))

# compare forecast to testing data
plt.plot(train, label='Train')
plt.plot(test, label='test')
plt.plot(forecast, label='Prediction')
plt.show()










## Segmentation comparison ##

* do any of our clusters correspond with rate code segments?
* if so, can we correct rate codes based on segments directly?
* if not, can we come up with a classifier to estimate rate code based on segment?

## KNN ##

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

# identify nearest neighbors
# KNN: K=10, default measure of distance (euclidean)
clf = KNeighborsClassifier(3)
clf.fit(X_train, y_train)
preds = clf.predict(X_test)

from sklearn.metrics import classification_report
print (classification_report(acts, preds))

# identify mode of neighbor classes

## Code Archive ##

### cross correlation ###

The proper way to compare for relationships between time series is by the cross-correlation function (*assuming stationarity*). Having the same length is not essential. 
The cross correlation at lag 0 just computes a correlation like doing the Pearson correlation estimate pairing the data at the identical time points. If they do have the same length as you are assuming, you will have exact T pairs where T is the number of time points for each series. 
Lag 1 cross correlation matches time t from series 1 with time t+1 in series 2. Note that here even though the series are the same length you only have T-2 pair as one point in the first series has no match in the second and one other point in the second series will not have a match from the first. Given these two series you can estimate the cross-correlation at several lags . 
If any of the cross correlations is statistically significantly different from 0 it will indicate a correlation between the two series.
see: https://stats.stackexchange.com/questions/29096/correlation-between-two-time-series, https://stats.stackexchange.com/questions/26842/correlating-volume-timeseries,
https://stackoverflow.com/questions/33171413/cross-correlation-time-lag-correlation-with-pandas#37215839

In [1]:
def crosscorr(y, X, lag=0):
    """ Lag-N cross correlation. 
    Parameters
    ----------
    lag : int, default 0
    y : pandas.Series object; independent variable
    X : pandas.Series object; matrix of dependent variables

    Returns
    ----------
    crosscorr : float
    """
    return X.corr(y.shift(lag))    